In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import os
from pyspark.sql.functions import col, lit, to_date, split, when, coalesce
from pyspark.sql.types import MapType, StringType

In [3]:
spark = SparkSession.builder \
                    .appName('tennis') \
                    .getOrCreate()

In [4]:
folder = "datasets/"
file_columns = []
files65 = []
files49= []

In [5]:
for filename in os.listdir(folder):

    file_path = os.path.join(folder, filename)
    
    df = spark.read.option("header", "true") \
                   .option("inferSchema", "true") \
                   .csv(file_path)
    
    #file_columns.append({filename:df.columns})
    file_columns.append(df.columns)
    if len(df.columns) == 65:
        files65.append(filename)
    else:
        files49.append(filename)

In [6]:
# Removing duplicates
file_columns = list(set(map(tuple, file_columns)))
file_columns = [list(item) for item in file_columns]
file_columns

[['tourney_id',
  'tourney_name',
  'surface',
  'draw_size',
  'tourney_level',
  'tourney_date',
  'match_num',
  'winner1_id',
  'winner2_id',
  'winner_seed',
  'winner_entry',
  'loser1_id',
  'loser2_id',
  'loser_seed',
  'loser_entry',
  'score',
  'best_of',
  'round',
  'winner1_name',
  'winner1_hand',
  'winner1_ht',
  'winner1_ioc',
  'winner1_age',
  'winner2_name',
  'winner2_hand',
  'winner2_ht',
  'winner2_ioc',
  'winner2_age',
  'loser1_name',
  'loser1_hand',
  'loser1_ht',
  'loser1_ioc',
  'loser1_age',
  'loser2_name',
  'loser2_hand',
  'loser2_ht',
  'loser2_ioc',
  'loser2_age',
  'winner1_rank',
  'winner1_rank_points',
  'winner2_rank',
  'winner2_rank_points',
  'loser1_rank',
  'loser1_rank_points',
  'loser2_rank',
  'loser2_rank_points',
  'minutes',
  'w_ace',
  'w_df',
  'w_svpt',
  'w_1stIn',
  'w_1stWon',
  'w_2ndWon',
  'w_SvGms',
  'w_bpSaved',
  'w_bpFaced',
  'l_ace',
  'l_df',
  'l_svpt',
  'l_1stIn',
  'l_1stWon',
  'l_2ndWon',
  'l_SvGms',
  

In [7]:
print(f"Number of different column sets: {len(file_columns)}")
print(f"Number of columns in the first set: {len(file_columns[0])}")
print(f"Number of columns in the second set: {len(file_columns[1])}")

Number of different column sets: 2
Number of columns in the first set: 65
Number of columns in the second set: 49


In [8]:
col1 = set(file_columns[0])
col2 = set(file_columns[1])
diff1 = col1 - col2 # Columns in Doubles files that are not in Singles files
diff2 = col2 - col1 # Columns in Singles files that are not in Doubles files
# ^ to get both

In [9]:
diff1

{'loser1_age',
 'loser1_hand',
 'loser1_ht',
 'loser1_id',
 'loser1_ioc',
 'loser1_name',
 'loser1_rank',
 'loser1_rank_points',
 'loser2_age',
 'loser2_hand',
 'loser2_ht',
 'loser2_id',
 'loser2_ioc',
 'loser2_name',
 'loser2_rank',
 'loser2_rank_points',
 'winner1_age',
 'winner1_hand',
 'winner1_ht',
 'winner1_id',
 'winner1_ioc',
 'winner1_name',
 'winner1_rank',
 'winner1_rank_points',
 'winner2_age',
 'winner2_hand',
 'winner2_ht',
 'winner2_id',
 'winner2_ioc',
 'winner2_name',
 'winner2_rank',
 'winner2_rank_points'}

In [10]:
diff2

{'loser_age',
 'loser_hand',
 'loser_ht',
 'loser_id',
 'loser_ioc',
 'loser_name',
 'loser_rank',
 'loser_rank_points',
 'winner_age',
 'winner_hand',
 'winner_ht',
 'winner_id',
 'winner_ioc',
 'winner_name',
 'winner_rank',
 'winner_rank_points'}

### Some should be added, others should be just changed (e.g. loser_age -> loser1_age)

In [11]:
# Files with 65 columns
files65

['atp_matches_doubles_2013.csv',
 'atp_matches_doubles_2005.csv',
 'atp_matches_doubles_2011.csv',
 'atp_matches_doubles_2009.csv',
 'atp_matches_doubles_2019.csv',
 'atp_matches_doubles_2020.csv',
 'atp_matches_doubles_2004.csv',
 'atp_matches_doubles_2006.csv',
 'atp_matches_doubles_2017.csv',
 'atp_matches_doubles_2016.csv',
 'atp_matches_doubles_2014.csv',
 'atp_matches_doubles_2008.csv',
 'atp_matches_doubles_2000.csv',
 'atp_matches_doubles_2001.csv',
 'atp_matches_doubles_2007.csv',
 'atp_matches_doubles_2015.csv',
 'atp_matches_doubles_2012.csv',
 'atp_matches_doubles_2018.csv',
 'atp_matches_doubles_2002.csv',
 'atp_matches_doubles_2010.csv',
 'atp_matches_doubles_2003.csv']

In [12]:
# Files with 49 columns
files49

['atp_matches_futures_2011.csv',
 'atp_matches_qual_chall_2005.csv',
 'atp_matches_1975.csv',
 'atp_matches_futures_2013.csv',
 'atp_matches_1987.csv',
 'atp_matches_2009.csv',
 'atp_matches_qual_chall_2001.csv',
 'atp_matches_qual_chall_2010.csv',
 'atp_matches_2021.csv',
 'atp_matches_1976.csv',
 'atp_matches_futures_1991.csv',
 'atp_matches_futures_2023.csv',
 'atp_matches_1998.csv',
 'atp_matches_1991.csv',
 'atp_matches_qual_chall_2020.csv',
 'atp_matches_qual_chall_1989.csv',
 'atp_matches_futures_2014.csv',
 'atp_matches_futures_1994.csv',
 'atp_matches_futures_1995.csv',
 'atp_matches_qual_chall_1981.csv',
 'atp_matches_qual_chall_1988.csv',
 'atp_matches_futures_2004.csv',
 'atp_matches_1999.csv',
 'atp_matches_futures_2010.csv',
 'atp_matches_1969.csv',
 'atp_matches_futures_2008.csv',
 'atp_matches_qual_chall_1991.csv',
 'atp_matches_2012.csv',
 'atp_matches_qual_chall_1997.csv',
 'atp_matches_1968.csv',
 'atp_matches_qual_chall_2013.csv',
 'atp_matches_1981.csv',
 'atp_matc

In [13]:
#HELENA

In [14]:
singles=spark.read.option("header","true").option("inferSchema","true") \
.csv('singles')

singles = singles.withColumn(
    "Origin",
    lit('Singles')
)

doubles=spark.read.option("header","true").option("inferSchema","true") \
.csv('doubles')

doubles = doubles.withColumn(
    "Origin",
    lit('Doubles')
)

futures=spark.read.option("header","true").option("inferSchema","true") \
.csv('futures')

futures = futures.withColumn(
    "Origin",
    lit('Futures')
)

qual_chal = spark.read.option("header","true").option("inferSchema","true") \
.csv('qual_chal')

qual_chal = qual_chal.withColumn(
    "Origin",
    lit('Qualificacao_Challenger')
)

amat = spark.read.option("header","true").option("inferSchema","true") \
.csv('atp_matches_amateur.csv')

amat = amat.withColumn(
    "Origin",
    lit('Amateur')
)

In [15]:
matches_completos = singles.unionAll(futures)
matches_completos = matches_completos.unionAll(qual_chal)
matches_completos=matches_completos.unionAll(amat)

In [16]:
colunas_unicas_doubles = list(set(doubles.columns) - set(matches_completos.columns))

for a in colunas_unicas_doubles:
    matches_completos = matches_completos.withColumn(
    a,
    lit(None)
)

sorted_columns = sorted(matches_completos.columns)
matches_completos = matches_completos.select(*sorted_columns)

In [17]:
colunas_unicas_matches_completos = list(set(matches_completos.columns) - set(doubles.columns))

for a in colunas_unicas_matches_completos:
    doubles = doubles.withColumn(
    a,
    lit(None)
)

In [18]:
sorted_columns = sorted(doubles.columns)
doubles = doubles.select(*sorted_columns)
matches_completos=matches_completos.unionAll(doubles)

In [19]:
len(matches_completos.columns)

82

In [20]:
matches_completos.select('tourney_id').show()
matches=matches_completos

+----------+
|tourney_id|
+----------+
|  1994-339|
|  1994-339|
|  1994-339|
|  1994-339|
|  1994-339|
|  1994-339|
|  1994-339|
|  1994-339|
|  1994-339|
|  1994-339|
|  1994-339|
|  1994-339|
|  1994-339|
|  1994-339|
|  1994-339|
|  1994-339|
|  1994-339|
|  1994-339|
|  1994-339|
|  1994-339|
+----------+
only showing top 20 rows



In [21]:
#LARA

In [22]:
# Listing all columns with the type 'void' and converting to int
void_columns = [col_name for col_name, data_type in matches.dtypes if data_type == 'void']
for col_name in void_columns:
    matches = matches.withColumn(col_name, matches[col_name].cast("int"))

In [23]:
# Converting to int
columns_to_convert = ["draw_size", "loser_id","loser_seed", "best_of", "minutes", "winner_seed",
    "w_ace", "w_df", "w_svpt", "w_1stIn", "w_1stWon", "w_2ndWon", "w_SvGms", "w_bpSaved", "w_bpFaced",
    "l_ace", "l_df", "l_svpt", "l_1stIn", "l_1stWon", "l_2ndWon", "l_SvGms", "l_bpSaved", "l_bpFaced",
    "winner_rank", "loser_rank", "winner_rank_points", "loser_rank_points", "winner2_hand"
]
for column in columns_to_convert:
    matches = matches.withColumn(column, col(column).cast("integer"))

In [24]:
matches = matches.withColumn("tourney_date", to_date(matches["tourney_date"].cast("string"), "yyyyMMdd"))
matches = matches.withColumn("winner_age", matches["winner_age"].cast("double"))

In [25]:
# Function to get scores for each set
def extrair_sets(scores):
    sets = scores.split(' ')  
    set_values = {}

    for i, set_score in enumerate(sets):
        try:
            set_values[f"set_{i+1}"] = int(set_score)
        except ValueError:
            set_values[f"set_{i+1}"] = set_score

    return set_values

extrair_sets_udf = F.udf(extrair_sets, MapType(StringType(), StringType()))

# New column with the scores for each set
matches = matches.withColumn("sets", extrair_sets_udf(col("score")))
for i in range(1, 6):  
    matches = matches.withColumn(f"set_{i}", when(col("sets").getItem(f"set_{i}").isNull(), None).otherwise(col("sets").getItem(f"set_{i}")))

matches = matches.drop("sets")

In [26]:
matches.select("set_1", "set_2", "set_3", "set_4", "set_5").show(1)

+-----+-----+-----+-----+-----+
|set_1|set_2|set_3|set_4|set_5|
+-----+-----+-----+-----+-----+
|  6-2|  6-2| NULL| NULL| NULL|
+-----+-----+-----+-----+-----+
only showing top 1 row



In [27]:
matches.filter(matches["set_5"].isNotNull()).select("set_5").show(10)

+-----+
|set_5|
+-----+
|  6-1|
|  8-6|
|  6-2|
|  6-4|
|  6-2|
|  8-6|
|  6-3|
|  RET|
|  6-2|
|  6-4|
+-----+
only showing top 10 rows



In [28]:
matches.printSchema()

root
 |-- Origin: string (nullable = false)
 |-- best_of: integer (nullable = true)
 |-- draw_size: integer (nullable = true)
 |-- l_1stIn: integer (nullable = true)
 |-- l_1stWon: integer (nullable = true)
 |-- l_2ndWon: integer (nullable = true)
 |-- l_SvGms: integer (nullable = true)
 |-- l_ace: integer (nullable = true)
 |-- l_bpFaced: integer (nullable = true)
 |-- l_bpSaved: integer (nullable = true)
 |-- l_df: integer (nullable = true)
 |-- l_svpt: integer (nullable = true)
 |-- loser1_age: double (nullable = true)
 |-- loser1_hand: string (nullable = true)
 |-- loser1_ht: integer (nullable = true)
 |-- loser1_id: integer (nullable = true)
 |-- loser1_ioc: string (nullable = true)
 |-- loser1_name: string (nullable = true)
 |-- loser1_rank: integer (nullable = true)
 |-- loser1_rank_points: integer (nullable = true)
 |-- loser2_age: double (nullable = true)
 |-- loser2_hand: string (nullable = true)
 |-- loser2_ht: integer (nullable = true)
 |-- loser2_id: integer (nullable = tr

In [29]:
doubles_columns = ['winner1_age', 'winner1_hand', 'winner1_ht', 'winner1_id', 'winner1_ioc', 'winner1_name', 'winner1_rank', 'winner1_rank_points', 'loser1_age', 'loser1_hand', 'loser1_ht', 'loser1_id', 'loser1_ioc', 'loser1_name', 'loser1_rank', 'loser1_rank_points']
singles_columns = ['winner_age', 'winner_hand', 'winner_ht', 'winner_id', 'winner_ioc', 'winner_name', 'winner_rank', 'winner_rank_points', 'loser_age', 'loser_hand', 'loser_ht', 'loser_id', 'loser_ioc', 'loser_name', 'loser_rank', 'loser_rank_points']

In [30]:
# Transferring the records in singles-related columns to doubles-related columns
for i in range(len(doubles_columns)):
    matches = matches.withColumn(f"{doubles_columns[i]}", coalesce(matches[f"{singles_columns[i]}"], matches[f"{doubles_columns[i]}"]))

In [31]:
# Dropping the columns not needed
matches = matches.drop(*singles_columns)

In [32]:
# 65 + Origin + 5 sets = 71
len(matches.columns)

71

In [33]:
# BEA

In [34]:
matches

DataFrame[Origin: string, best_of: int, draw_size: int, l_1stIn: int, l_1stWon: int, l_2ndWon: int, l_SvGms: int, l_ace: int, l_bpFaced: int, l_bpSaved: int, l_df: int, l_svpt: int, loser1_age: double, loser1_hand: string, loser1_ht: int, loser1_id: int, loser1_ioc: string, loser1_name: string, loser1_rank: int, loser1_rank_points: int, loser2_age: double, loser2_hand: string, loser2_ht: int, loser2_id: int, loser2_ioc: string, loser2_name: string, loser2_rank: int, loser2_rank_points: int, loser_entry: string, loser_seed: int, match_num: int, minutes: int, round: string, score: string, surface: string, tourney_date: date, tourney_id: string, tourney_level: string, tourney_name: string, w_1stIn: int, w_1stWon: int, w_2ndWon: int, w_SvGms: int, w_ace: int, w_bpFaced: int, w_bpSaved: int, w_df: int, w_svpt: int, winner1_age: double, winner1_hand: string, winner1_ht: int, winner1_id: int, winner1_ioc: string, winner1_name: string, winner1_rank: int, winner1_rank_points: int, winner2_age: 

In [84]:
from pyspark.sql.functions import count
print("-------------------------------------------")
# Count null values in 'Origin' column
null_count = matches.select('Origin').where(col('Origin').isNull()).count()
# Count total values set_1
total_values = matches.select('set_1').count()
print(f"Total null values in 'Origin' column: {null_count}")
print(f"Total values in 'set_1' column: {total_values}")
print("-------------------------------------------")




# Only use until winnner_seed, set_1, set_2, etc... were throwing errors
relevant_columns = matches.columns[: matches.columns.index('winner_seed') + 1]  # Add 1 for inclusive indexing

# Calculate null values counts
null_columns = []
for column in relevant_columns:
    null_value_counts = matches.select(column).where(col(column).isNull()).count()
    # Percentage of null values
    percentage = (null_value_counts / matches.select(column).count()) * 100
    # Print the results
    print(f"Column name: {column} has a total of {null_value_counts} null values and percentage of nulls: {percentage:.2f}%")
    
    
    # If percentage of nulls superior to 75% delete column?
    if percentage >= 75:
        null_columns.append(column)

print("-------------------------------------------")
print("Columns with null percentage superior to 75%")
for column_null in null_columns:
    print(f"Column name: {column_null}")

-------------------------------------------
Total null values in 'Origin' column: 0
Total values in 'set_1' column: 935359
-------------------------------------------
Column name: Origin has a total of 0 null values and percentage of nulls: 0.00%
Column name: best_of has a total of 0 null values and percentage of nulls: 0.00%
Column name: draw_size has a total of 2477 null values and percentage of nulls: 0.26%
Column name: l_1stIn has a total of 730169 null values and percentage of nulls: 78.06%
Column name: l_1stWon has a total of 730169 null values and percentage of nulls: 78.06%
Column name: l_2ndWon has a total of 730169 null values and percentage of nulls: 78.06%
Column name: l_SvGms has a total of 730170 null values and percentage of nulls: 78.06%
Column name: l_ace has a total of 730169 null values and percentage of nulls: 78.06%
Column name: l_bpFaced has a total of 730169 null values and percentage of nulls: 78.06%
Column name: l_bpSaved has a total of 730169 null values and p

In [ ]:
# Add some data correlations